First of all, this notebook needs a couple of third-party 'modules' to work properly. These are used to draw animated graphics inside this Jupyter notebook. Execute the following cell by selecting it and pressing <kbd>Shift</kbd>+<kbd>Enter</kbd>.

In [ ]:
import sys
!{sys.executable} -m pip install ipycanvas
!{sys.executable} -m pip install ipyevents

In [ ]:
# Change the cell width to better fit the graphics on the screen.
from IPython.core.display import HTML
from IPython import display
display.display(HTML("<style>.container { width:100% !important; }</style>"))

The code we're about to import is a sort-of-detailed physics simulation of a car, along with some graphics code to animate the output of the simulation. The idea is that we will be writing a function which drives the car by setting its steering angle, throttle and brake.

The control function we provide takes two arguments. These are objects which the simulation passes to our function in every time-step of the simulation.

The <code>sim_params</code> argument contains all of the fixed information the simulation uses: the mass of the car, how much torque the engine produces, the amount of friction the tyres can generate, how many simulation steps to run per second, and so on.

The <code>sim_state</code> argument contains all of the dynamic information which is calculated by the simulation as it runs: the position and velocity of the car, the engine RPM, the car's heading, and so on.

Each simulation time-step, our control function is called. We can do anything we like in our function, but in the end we have to return three numbers: the steering rack angle (in radians), the throttle fraction (0 to 1) and the brake fraction (0 to 1).

The objective is to drive the car through a series of waypoints and stop on the last one. The distance and bearing to the objective are available in <code>sim_state.objective.distance_m</code> and <code>sim_state.objective.bearing_rad</code> respectively, but if you want to get fancy you can access the positions of all the waypoints in <code>sim_params.objective</code>.

The cell below imports the code which runs the car simulation. Execute it by selecting it and pressing <kbd>Shift</kbd>+<kbd>Enter</kbd>.

In [ ]:
from car_sim.widget import get_car_sim_widget

In the next cell is an example of an extremely simple control function that doesn't work very well at all, but still manages to complete the example course below.

The function can do anything you like, as long as long as it returns a 3-tuple containing numbers corresponding to the control variables described above.

In [ ]:
def ControlFunction(sim_params, sim_state):
    # Steering angle, throttle, brake.
    return (sim_state.objective.bearing_rad,
            sim_state.objective.distance_m * (0.1 if abs(sim_state.objective.bearing_rad) < 0.2 else 0.005),
            1 if sim_state.objective.distance_m < 10 else 0)

To run the simulation, execucte the following cell. You can adjust the width, height and scale factor to suit your preferences.

Note that we're passing the <code>ControlFunction</code> defined above to the simulation widget; you'll need to re-execute this cell each time you change the control function.

Run the simulation and see how the controller performs.

In [ ]:
get_car_sim_widget(ControlFunction,
                   width=1250, height=700, scale=0.5,
                   init_pos=[0.0, -20.0],
                   objective=[[-20.0, 20.0], [20.0, 20.0], [20.0, -20.0], [-20.0, 0.0]])

Here's a slightly more complex (but rather slow) example.

In [ ]:
import math
def ControlFunction(sim_params, sim_state):
    steering_gain = 0.3
    steering_angle = sim_state.objective.bearing_rad * steering_gain
    
    brake = 1 if sim_state.objective.distance_m < 5 else 0
    
    vehicle_speed = math.sqrt(sim_state.vehicle.velocity_m_s[0]**2 + sim_state.vehicle.velocity_m_s[1]**2)
    target_speed = 10
    throttle_gain = 0.05
    throttle = (target_speed - vehicle_speed) * throttle_gain * (1 - brake)
    return (steering_angle, throttle, brake)

Try and design your own controller to get the best time on the course above. Ideally it should be general enough that it still works well on other courses, though – you can adjust the number and location of the objective points to design your own if you like.